In [1]:
ROOT = "../../"
import sys
sys.path.append(ROOT)
#
from src.constants import MOVIES_PATH, RATINGS_PATH, USERS_PATH
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Data

## Users

In [2]:
# 1) Read lines and split them by "::"
# 2) Select the columns and cast them to the correct types
ocupations = [
    "other",
	"academic/educator",
	"artist",
	"clerical/admin",
	"college/grad student",
	"customer service",
	"doctor/health care",
	"executive/managerial",
	"farmer",
	"homemaker",
	"K-12 student",
	"lawyer",
	"programmer",
	"retired",
	"sales/marketing",
	"scientist",
	"self-employed",
	"technician/engineer",
	"tradesman/craftsman",
	"unemployed",
	"writer",
]
occupation_map = {i: occupation for i, occupation in enumerate(ocupations)}
users_df = pl.scan_csv(ROOT + USERS_PATH, has_header=False, truncate_ragged_lines=True, encoding="utf8-lossy").select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("UserID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("Gender"),
    pl.col("column_1").list.get(2).alias("Age").cast(pl.Int32),
    pl.col("column_1").list.get(3).alias("Occupation").cast(pl.Int32).map_dict(occupation_map),
    pl.col("column_1").list.get(4).alias("Zip-code"),
])

users_df.head().collect()

/var/folders/16/2kmt14y56rqd2xkgkp53tr080000gn/T/ipykernel_87190/3861513786.py:33: DeprecationWarning: `map_dict` is deprecated. It has been renamed to `replace`. The default behavior has changed to keep any values not present in the mapping unchanged. Pass `default=None` to keep existing behavior.
  pl.col("column_1").list.get(3).alias("Occupation").cast(pl.Int32).map_dict(occupation_map),


UserID,Gender,Age,Occupation,Zip-code
i32,str,i32,str,str
1,"""F""",1,"""K-12 student""","""48067"""
2,"""M""",56,"""self-employed""","""70072"""
3,"""M""",25,"""scientist""","""55117"""
4,"""M""",45,"""executive/managerial""","""02460"""
5,"""M""",25,"""writer""","""55455"""


## Movies


In [3]:
# 1) Read lines and split them by "::"
# 2) Select the columns and cast them to the correct types
# 3) Extract the year from the title column
# 4) If year is not null, remove it from the title
#   If the title is "Toy Story (1995)", the title should be "Toy Story" and the year should be 1995. 
#   If the title does not have a year, the year should be null.
movies_df = pl.scan_csv(ROOT + MOVIES_PATH, has_header=False, truncate_ragged_lines=True, encoding="utf8-lossy").select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("MovieID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("Title"),
    pl.col("column_1").list.get(2).str.split("|").alias("Genres")
]).with_columns([
    pl.col("Title").str.extract(r"\((\d{4})\)$").alias("Year").cast(pl.Int32),
]).with_columns([
    # If Year is not null, remove it from the title
    pl.when(pl.col("Year").is_null()).then(pl.col("Title")).otherwise(
        pl.col("Title").str.slice(0, pl.col("Title").str.find(" \((\d{4})\)$"))
    ).alias("Title")
])
# 4) Get possible genres
genres = movies_df.select(pl.col("Genres").explode().unique()).collect()
# 5) Make dummy variable for each genre
movies_df = movies_df.with_columns([
    pl.col("Genres").list.contains(genre[0]).alias(f"Is{genre[0]}")
    for genre in genres.rows()
])

In [4]:
movies_df.collect()

MovieID,Title,Genres,Year,IsChildren's,IsSci-Fi,IsWar,IsFantasy,IsAction,IsAdventure,IsHorror,IsFilm-Noir,IsRomance,IsThriller,IsAnimation,IsCrime,IsWestern,IsDocumentary,IsMusical,IsNone,IsMystery,IsDrama,IsComedy
i32,str,list[str],i32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
1,"""Toy Story""","[""Animation"", ""Children's"", ""Comedy""]",1995,true,false,false,false,false,false,false,false,false,false,true,false,false,false,false,null,false,false,true
2,"""Jumanji""","[""Adventure"", ""Children's"", ""Fantasy""]",1995,true,false,false,true,false,true,false,false,false,false,false,false,false,false,false,null,false,false,false
3,"""Grumpier Old Men""","[""Comedy"", ""Romance""]",1995,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,null,false,false,true
4,"""Waiting to Exhale""","[""Comedy"", ""Drama""]",1995,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,true,true
5,"""Father of the Bride Part II""","[""Comedy""]",1995,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3948,"""Meet the Parents""","[""Comedy""]",2000,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,false,true
3949,"""Requiem for a Dream""","[""Drama""]",2000,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,true,false
3950,"""Tigerland""","[""Drama""]",2000,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,true,false


## Ratings

In [5]:
# 1) Read lines and split them by "::"
# 2) Select columns and cast them to the correct type
ratings_df = pl.scan_csv(ROOT + RATINGS_PATH, has_header=False).select([
    pl.col("column_1").str.split("::")
]).select([
    pl.col("column_1").list.get(0).alias("UserID").cast(pl.Int32),
    pl.col("column_1").list.get(1).alias("MovieID").cast(pl.Int32),
    pl.col("column_1").list.get(2).alias("Rating").cast(pl.Int32),
    pl.col("column_1").list.get(3).alias("Timestamp").cast(pl.Int32)
])

ratings_df.head().collect()

UserID,MovieID,Rating,Timestamp
i32,i32,i32,i32
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291


## Join

In [6]:
# Join dataframes
df = ratings_df.join(movies_df, on="MovieID").join(users_df, on="UserID")
df.head().collect()

UserID,MovieID,Rating,Timestamp,Title,Genres,Year,IsChildren's,IsSci-Fi,IsWar,IsFantasy,IsAction,IsAdventure,IsHorror,IsFilm-Noir,IsRomance,IsThriller,IsAnimation,IsCrime,IsWestern,IsDocumentary,IsMusical,IsNone,IsMystery,IsDrama,IsComedy,Gender,Age,Occupation,Zip-code
i32,i32,i32,i32,str,list[str],i32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,i32,str,str
1,1193,5,978300760,"""One Flew Over the Cuckoo's Nes…","[""Drama""]",1975,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,true,false,"""F""",1,"""K-12 student""","""48067"""
1,661,3,978302109,"""James and the Giant Peach""","[""Animation"", ""Children's"", ""Musical""]",1996,true,false,false,false,false,false,false,false,false,false,true,false,false,false,true,null,false,false,false,"""F""",1,"""K-12 student""","""48067"""
1,914,3,978301968,"""My Fair Lady""","[""Musical"", ""Romance""]",1964,false,false,false,false,false,false,false,false,true,false,false,false,false,false,true,null,false,false,false,"""F""",1,"""K-12 student""","""48067"""
1,3408,4,978300275,"""Erin Brockovich""","[""Drama""]",2000,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,true,false,"""F""",1,"""K-12 student""","""48067"""
1,2355,5,978824291,"""Bug's Life""",null,null,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,null,false,false,false,"""F""",1,"""K-12 student""","""48067"""
